In [1]:
import os
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

from functools import partial

import time
import scipy.stats as stats
import seaborn as sns
import arviz as az

#import constants

from tqdm import tqdm
from tqdm.auto import tqdm
tqdm.pandas()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

pd.set_option('display.max_columns', None)

/opt/anaconda3/envs/Boda_2/lib/python3.7/site-packages/tqdm/std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [36]:
boda2_df = pd.read_csv('/Users/castrr/Dropbox (JAX)/Datasets/BODA2/BODA2_MPRA_results_pred_v3.txt', sep=" ", low_memory=False)

In [23]:
contributions_df = pd.read_csv('/Users/castrr/Dropbox (JAX)/Datasets/BODA2/BODA2_MPRA_results_no_controls_pred_contributions.txt', sep=" ", low_memory=False)

In [37]:
boda2_df

,ID,sequence,K562_pred,HepG2_pred,SKNSH_pred,K562_pred_aggreg,HepG2_pred_aggreg,SKNSH_pred_aggreg,project,K562_signal,K562_lfcSE,HepG2_signal,HepG2_lfcSE,SKNSH_signal,SKNSH_lfcSE,method,target_cell,round,max_cell_pred,max_cell_signal,max_non_target_cell,max_non_target_lfc,min_non_target_cell,min_non_target_lfc,target_cell_lfc
0,20211210_200256__612928__5::fsp_uc__hepg2__0,CAATCTGTTCCCACCATCAGGTTACTGGTTAATCATTAACCAGGGC...,-0.055265,14.231705,0.080212,0.140002,14.336414,0.227027,BODA:hepg2,2.520816,0.316205,5.287727,0.309186,0.605723,0.208455,fsp_uc,hepg2,0,hepg2,hepg2,k562,2.520816,sknsh,0.605723,5.287727
1,20211210_200256__612928__12::fsp_uc__hepg2__0,GGAGCTCTTTCAATGCCAGGCAGGCCCGTGGTGCAGTAAAGGCCTG...,-0.019169,19.914394,0.777519,-0.011601,18.561650,0.767500,BODA:hepg2,0.609272,0.285060,6.402153,0.301327,0.271218,0.340279,fsp_uc,hepg2,0,hepg2,hepg2,k562,0.609272,sknsh,0.271218,6.402153
2,20211210_200256__612928__47::fsp_uc__hepg2__0,GGTTCAAAGTCCAGTCACAGTCCAAAGTCCACTCTTTGTCCAAGAG...,-0.146010,12.227689,-0.400996,-0.034775,10.825762,-0.337030,BODA:hepg2,1.486130,0.271874,4.276053,0.366801,-0.578350,0.241865,fsp_uc,hepg2,0,hepg2,hepg2,k562,1.486130,sknsh,-0.578350,4.276053
3,20211210_200256__612928__69::fsp_uc__hepg2__0,TAATAGTATGTCATCGTAATATGCGGACTTTGAACCGTGTACCAGG...,1.216181,18.812680,1.589409,1.416824,17.606884,1.713512,BODA:hepg2,2.440630,0.219559,5.615395,0.313984,1.063669,0.230603,fsp_uc,hepg2,0,hepg2,hepg2,k562,2.440630,sknsh,1.063669,5.615395
4,20211210_200256__612928__93::fsp_uc__hepg2__0,AACACGGGCACGGGGGGGCGCTGTTTGAGTTTAAAAACGATCTAGG...,1.832291,18.612158,1.411221,2.409507,17.280722,2.059474,BODA:hepg2,3.765216,0.570148,4.683140,0.612207,0.171043,2.165178,fsp_uc,hepg2,0,hepg2,hepg2,k562,3.765216,sknsh,0.171043,4.683140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120052,ORF54316:2538:1300-1500,CTCGGAGTTCAAGAAAATCATCTCACACGGCCGTCTCAGACCCTTC...,-0.119510,1.166959,-0.200713,-0.158865,1.282273,-0.275202,MPRA:ORF_negCtrl,-0.315267,0.384919,0.591628,0.363142,0.068697,0.275408,NaN,NaN,0,hepg2,hepg2,NaN,NaN,NaN,NaN,NaN
120053,ORF53130:1455:400-600,GCAATCCTACCCAGACAGATGCAGACAGCGACCTGGTGGGGGATGT...,-0.301822,-0.577524,-0.384271,-0.247987,-0.644892,-0.367910,MPRA:ORF_negCtrl,-0.134821,0.178379,-0.359782,0.261176,-0.022281,0.200300,NaN,NaN,0,k562,sknsh,NaN,NaN,NaN,NaN,NaN
120054,ORF55877:2391:1900-2100,CCTATGCTGCCTCCTATGCCCTTGTCATCATTGCCATTTTTGAGCT...,1.219841,1.099753,0.645146,0.524327,0.483457,0.248783,MPRA:ORF_negCtrl,0.646673,0.259927,0.489900,0.190735,0.462366,0.111966,NaN,NaN,0,k562,k562,NaN,NaN,NaN,NaN,NaN
120055,ORF52950:2193:400-600,AGCCACAACAAAACAGCCAGTTAATGTGGAAACAAGGTCGACAACT...,0.132379,0.718228,0.953675,0.085746,0.563376,0.668541,MPRA:ORF_negCtrl,-0.061676,0.171760,-0.014411,0.199712,0.239693,0.166026,NaN,NaN,0,sknsh,sknsh,NaN,NaN,NaN,NaN,NaN


In [7]:
contributions_df

,ID,sequence,K562_pred,HepG2_pred,SKNSH_pred,K562_pred_aggreg,HepG2_pred_aggreg,SKNSH_pred_aggreg,project,K562_signal,K562_lfcSE,HepG2_signal,HepG2_lfcSE,SKNSH_signal,SKNSH_lfcSE,method,target_cell,round,max_cell_pred,max_cell_signal,max_non_target_cell,max_non_target_lfc,min_non_target_cell,min_non_target_lfc,target_cell_lfc,contrib_K562,contrib_HepG2,contrib_SKNSH
0,20211210_200256__612928__5::fsp_uc__hepg2__0,CAATCTGTTCCCACCATCAGGTTACTGGTTAATCATTAACCAGGGC...,-0.055265,14.231705,0.080212,0.140002,14.336414,0.227027,BODA:hepg2,2.520816,0.316205,5.287727,0.309186,0.605723,0.208455,fsp_uc,hepg2,0,hepg2,hepg2,k562,2.520816,sknsh,0.605723,5.287727,[ 2.21043210e-02 4.64022532e-03 1.00994157e-...,[ 1.1422859e-01 2.5873125e-01 2.2765625e-01 ...,[ 0.004331928 0.07783466 0.06386908 ...
1,20211210_200256__612928__12::fsp_uc__hepg2__0,GGAGCTCTTTCAATGCCAGGCAGGCCCGTGGTGCAGTAAAGGCCTG...,-0.019169,19.914394,0.777519,-0.011601,18.561650,0.767500,BODA:hepg2,0.609272,0.285060,6.402153,0.301327,0.271218,0.340279,fsp_uc,hepg2,0,hepg2,hepg2,k562,0.609272,sknsh,0.271218,6.402153,[-0.00856874 -0.012687626 0.03335264 ...,[-2.79185679e-02 2.33078538e-03 2.57074833e-...,[-0.017693618 -0.015454747 0.01019545 ...
2,20211210_200256__612928__47::fsp_uc__hepg2__0,GGTTCAAAGTCCAGTCACAGTCCAAAGTCCACTCTTTGTCCAAGAG...,-0.146010,12.227689,-0.400996,-0.034775,10.825762,-0.337030,BODA:hepg2,1.486130,0.271874,4.276053,0.366801,-0.578350,0.241865,fsp_uc,hepg2,0,hepg2,hepg2,k562,1.486130,sknsh,-0.578350,4.276053,[-5.54974526e-02 -2.83153541e-02 -1.36791170e-...,[ 0.041962117 0.2561933 0.058943637 0....,[-4.9518015e-02 -2.5722878e-02 -4.6560772e-02 ...
3,20211210_200256__612928__69::fsp_uc__hepg2__0,TAATAGTATGTCATCGTAATATGCGGACTTTGAACCGTGTACCAGG...,1.216181,18.812680,1.589409,1.416824,17.606884,1.713512,BODA:hepg2,2.440630,0.219559,5.615395,0.313984,1.063669,0.230603,fsp_uc,hepg2,0,hepg2,hepg2,k562,2.440630,sknsh,1.063669,5.615395,[ 0.16365409 0.0731409 0.114835456 ...,[ 1.40735984e-01 1.30295590e-01 1.24457464e-...,[ 1.11630686e-01 9.51312631e-02 1.00722231e-...
4,20211210_200256__612928__93::fsp_uc__hepg2__0,AACACGGGCACGGGGGGGCGCTGTTTGAGTTTAAAAACGATCTAGG...,1.832291,18.612158,1.411221,2.409507,17.280722,2.059474,BODA:hepg2,3.765216,0.570148,4.683140,0.612207,0.171043,2.165178,fsp_uc,hepg2,0,hepg2,hepg2,k562,3.765216,sknsh,0.171043,4.683140,[ 0.017253371 0.03226251 -0.008712402 ...,[ 0.04122336 0.040660232 -0.0046606357 ...,[ 0.05285579 0.042130962 -0.035606854 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117895,chr4:49653500-49653700,AATGGAATGGAATGGAATGGAAATGAATGGAATGGAATAACATGCA...,-0.259550,-0.350268,1.980883,-0.238864,-0.339671,1.852794,BODA:genomic,-0.358920,0.247093,-0.248105,0.266864,-0.112114,0.191480,genome_scan,sknsh,0,sknsh,sknsh,hepg2,-0.248105,k562,-0.358920,-0.112114,[ 1.57648958e-02 2.40496714e-02 1.48133254e-...,[ 0.04095571 0.023086062 -0.0193771 ...,[ 0.08480706 0.0433614 -0.00055749423 ...
117896,chr20:31056400-31056600,TGGAATGGAATGAAGGGGAATGGAATGGAATGGAATGCAATGGATT...,-0.139882,-0.331289,2.217611,-0.154154,-0.320628,1.972111,BODA:genomic,-0.246672,0.299501,0.005176,0.313814,0.564290,0.239431,genome_scan,sknsh,0,sknsh,sknsh,hepg2,0.005176,k562,-0.246672,0.564290,[ 1.83610339e-02 1.29304640e-03 1.54471565e-...,[ 1.05690164e-02 2.45974623e-02 2.49181334e-...,[ 4.41843010e-02 3.73021774e-02 6.50241747e-...
117897,chr4:49144600-49144800,TTCCATTCCATTGCATTCCATACCATTCCATTCCATTCTATTCCAT...,-0.148705,0.614329,3.502213,-0.106589,0.443236,3.642229,BODA:genomic,2.172089,0.276453,2.214319,0.229850,2.428773,0.112830,genome_scan,sknsh,0,sknsh,sknsh,hepg2,2.214319,k562,2.172089,2.428773,[ 2.16334686e-02 -2.50381930e-03 -3.50337662e-...,[ 7.59168947e-03 -3.31764692e-03 -5.40563948e-...,[ 0.028185885 -0.029818058 -0.0979394 0....
117898,chr11:82436300-82436500,TCCAGACATAATAACAGGAATGTATACTTCCTGGCCATCAGAGCCT...,-0.138973,0.152324,2.755640,-0.078967,0.112190,1.910803,BODA:genomic,0.113179,0.47310

In [31]:
no_controls_df = boda2_df[boda2_df['method'].notnull()].copy().reset_index(drop=True)

In [32]:
no_controls_df['mean_non_target_lfc'] = 0.5*(no_controls_df['max_non_target_lfc'] + no_controls_df['min_non_target_lfc'])
no_controls_df['OverMax'] = no_controls_df['target_cell_lfc'] - no_controls_df['max_non_target_lfc']
no_controls_df['OverMean'] = no_controls_df['target_cell_lfc'] - no_controls_df['mean_non_target_lfc']

In [38]:
to_attach_df = no_controls_df[["ID", 'mean_non_target_lfc', 'OverMean', 'OverMax']].copy()
boda2_df = boda2_df.merge(to_attach_df, how='outer', on='ID').copy()

In [39]:
to_attach_df = contributions_df[['ID', 'contrib_K562', 'contrib_HepG2', 'contrib_SKNSH']].copy()
boda2_df = boda2_df.merge(to_attach_df, how='outer', on='ID').copy()

In [40]:
boda2_df

,ID,sequence,K562_pred,HepG2_pred,SKNSH_pred,K562_pred_aggreg,HepG2_pred_aggreg,SKNSH_pred_aggreg,project,K562_signal,K562_lfcSE,HepG2_signal,HepG2_lfcSE,SKNSH_signal,SKNSH_lfcSE,method,target_cell,round,max_cell_pred,max_cell_signal,max_non_target_cell,max_non_target_lfc,min_non_target_cell,min_non_target_lfc,target_cell_lfc,mean_non_target_lfc,OverMean,OverMax,contrib_K562,contrib_HepG2,contrib_SKNSH
0,20211210_200256__612928__5::fsp_uc__hepg2__0,CAATCTGTTCCCACCATCAGGTTACTGGTTAATCATTAACCAGGGC...,-0.055265,14.231705,0.080212,0.140002,14.336414,0.227027,BODA:hepg2,2.520816,0.316205,5.287727,0.309186,0.605723,0.208455,fsp_uc,hepg2,0,hepg2,hepg2,k562,2.520816,sknsh,0.605723,5.287727,1.563270,3.724457,2.766911,[ 2.21043210e-02 4.64022532e-03 1.00994157e-...,[ 1.1422859e-01 2.5873125e-01 2.2765625e-01 ...,[ 0.004331928 0.07783466 0.06386908 ...
1,20211210_200256__612928__12::fsp_uc__hepg2__0,GGAGCTCTTTCAATGCCAGGCAGGCCCGTGGTGCAGTAAAGGCCTG...,-0.019169,19.914394,0.777519,-0.011601,18.561650,0.767500,BODA:hepg2,0.609272,0.285060,6.402153,0.301327,0.271218,0.340279,fsp_uc,hepg2,0,hepg2,hepg2,k562,0.609272,sknsh,0.271218,6.402153,0.440245,5.961907,5.792880,[-0.00856874 -0.012687626 0.03335264 ...,[-2.79185679e-02 2.33078538e-03 2.57074833e-...,[-0.017693618 -0.015454747 0.01019545 ...
2,20211210_200256__612928__47::fsp_uc__hepg2__0,GGTTCAAAGTCCAGTCACAGTCCAAAGTCCACTCTTTGTCCAAGAG...,-0.146010,12.227689,-0.400996,-0.034775,10.825762,-0.337030,BODA:hepg2,1.486130,0.271874,4.276053,0.366801,-0.578350,0.241865,fsp_uc,hepg2,0,hepg2,hepg2,k562,1.486130,sknsh,-0.578350,4.276053,0.453890,3.822163,2.789924,[-5.54974526e-02 -2.83153541e-02 -1.36791170e-...,[ 0.041962117 0.2561933 0.058943637 0....,[-4.9518015e-02 -2.5722878e-02 -4.6560772e-02 ...
3,20211210_200256__612928__69::fsp_uc__hepg2__0,TAATAGTATGTCATCGTAATATGCGGACTTTGAACCGTGTACCAGG...,1.216181,18.812680,1.589409,1.416824,17.606884,1.713512,BODA:hepg2,2.440630,0.219559,5.615395,0.313984,1.063669,0.230603,fsp_uc,hepg2,0,hepg2,hepg2,k562,2.440630,sknsh,1.063669,5.615395,1.752150,3.863246,3.174765,[ 0.16365409 0.0731409 0.114835456 ...,[ 1.40735984e-01 1.30295590e-01 1.24457464e-...,[ 1.11630686e-01 9.51312631e-02 1.00722231e-...
4,20211210_200256__612928__93::fsp_uc__hepg2__0,AACACGGGCACGGGGGGGCGCTGTTTGAGTTTAAAAACGATCTAGG...,1.832291,18.612158,1.411221,2.409507,17.280722,2.059474,BODA:hepg2,3.765216,0.570148,4.683140,0.612207,0.171043,2.165178,fsp_uc,hepg2,0,hepg2,hepg2,k562,3.765216,sknsh,0.171043,4.683140,1.968129,2.715011,0.917925,[ 0.017253371 0.03226251 -0.008712402 ...,[ 0.04122336 0.040660232 -0.0046606357 ...,[ 0.05285579 0.042130962 -0.035606854 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120052,ORF54316:2538:1300-1500,CTCGGAGTTCAAGAAAATCATCTCACACGGCCGTCTCAGACCCTTC...,-0.119510,1.166959,-0.200713,-0.158865,1.282273,-0.275202,MPRA:ORF_negCtrl,-0.315267,0.384919,0.591628,0.363142,0.068697,0.275408,NaN,NaN,0,hepg2,hepg2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120053,ORF53130:1455:400-600,GCAATCCTACCCAGACAGATGCAGACAGCGACCTGGTGGGGGATGT...,-0.301822,-0.577524,-0.384271,-0.247987,-0.644892,-0.367910,MPRA:ORF_negCtrl,-0.134821,0.178379,-0.359782,0.261176,-0.022281,0.200300,NaN,NaN,0,k562,sknsh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120054,ORF55877:2391:1900-2100,CCTATGCTGCCTCCTATGCCCTTGTCATCATTGCCATTTTTGAGCT...,1.219841,1.099753,0.645146,0.524327,0.483457,0.248783,MPRA:ORF_negCtrl,0.646673,0.259927,0.489900,0.190735,0.462366,0.111966,NaN,NaN,0,k562,k562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120055,ORF52950:2193:400-600,AGCCACAACAAAACAGCCAGTTAATGTGGAAACAAGGTCGACAACT...,0.132379,0.718228,0.953675,0.085746,0.563376,0.668541,MPRA:ORF_negCtrl,-0.061676,0.171760,-0.014411,0.199712,0.239693,0.166026,NaN,NaN,0,sknsh,sknsh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
K562  = pd.read_table('/Users/castrr/Dropbox (JAX)/analysis_2022718/MPRAmodel_output/results/OL46_K562_20220720.out', sep='\t', header=0, index_col='ID').reset_index(drop=False)
HepG2 = pd.read_table('/Users/castrr/Dropbox (JAX)/analysis_2022718/MPRAmodel_output/results/OL46_HepG2_20220720.out', sep='\t', header=0, index_col='ID').reset_index(drop=False)
SKNSH = pd.read_table('/Users/castrr/Dropbox (JAX)/analysis_2022718/MPRAmodel_output/results/OL46_SKNSH_20220720.out', sep='\t', header=0, index_col='ID').reset_index(drop=False)

In [52]:
K562

,ID,SNP,chr,start,stop,pos,ref_allele,alt_allele,allele,window,strand,project,haplotype,ctrl_exp,DNA_mean,ctrl_mean,exp_mean,log2FoldChange,lfcSE,stat,pvalue,padj
0,1:10437778:C:T:R:wC,NaN,chr1,10377621,10377820,NaN,NaN,NaN,NaN,NaN,+,MPRA:ctrl_pos,NaN,MPRA:ctrl_pos,828.000000,510.066273,3982.985875,2.966183,0.352146,8.423174,3.664205e-17,1.381649e-16
1,1:10451799:C:T:R:wC,NaN,chr1,10391643,10391842,NaN,NaN,NaN,NaN,NaN,+,MPRA:ctrl_neg,NaN,MPRA:ctrl_neg,1007.333333,619.517011,663.645365,0.095328,0.180777,0.527324,5.979684e-01,6.724548e-01
2,1:110198727:C:G:R:wC,NaN,chr1,109656006,109656205,NaN,NaN,NaN,NaN,NaN,+,MPRA:ctrl_pos,NaN,MPRA:ctrl_pos,62.333333,38.926483,3945.704881,6.666020,0.357260,18.658744,1.072219e-77,2.227386e-75
3,1:110881742:NA:NA,NaN,chr1,110339021,110339220,NaN,NaN,NaN,NaN,NaN,-,MPRA:ctrl_pos,NaN,MPRA:ctrl_pos,1296.000000,800.051571,23837.898516,4.896945,0.402289,12.172713,4.343984e-34,3.236403e-33
4,1:111838694:C:T:R:wC,NaN,chr1,111295973,111296172,NaN,NaN,NaN,NaN,NaN,+,MPRA:ctrl_pos,NaN,MPRA:ctrl_pos,622.666667,383.147871,22963.814407,5.905227,0.405517,14.562203,4.885725e-48,8.967351e-47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120052,X:24893812:NA:NA,NaN,chrX,24875596,24875795,NaN,NaN,NaN,NaN,NaN,+,MPRA:ctrl_neg,NaN,MPRA:ctrl_neg,1984.333333,1216.543339,1320.653224,0.120669,0.183022,0.659316,5.096929e-01,5.914998e-01
120053,X:39110157:NA:NA,NaN,chrX,39250805,39251004,NaN,NaN,NaN,NaN,NaN,+,MPRA:ctrl_neg,NaN,MPRA:ctrl_neg,1164.666667,718.753085,911.977644,0.343890,0.165054,2.083493,3.720627e-02,6.146985e-02
120054,X:49154578:NA:NA,NaN,chrX,49298001,49298200,NaN,NaN,NaN,NaN,NaN,+,MPRA:ctrl_neg,NaN,MPRA:ctrl_neg,1496.333333,919.641399,1002.931530,0.129982,0.177481,0.732369,4.639436e-01,5.480029e-01
120055,X:64815540:NA:NA,NaN,chrX,65595561,65595760,NaN,NaN,NaN,NaN,NaN,+,MPRA:ctrl_pos,NaN,MPRA:ctrl_pos,2674.333333,1642.172830,37378.172560,4.508452,0.424550,10.619364,2.422050e-26,1.261783e-25


In [54]:
to_attach_df = pd.concat([K562.loc[:,('ID', 'ctrl_mean', 'exp_mean')],
                          HepG2.loc[:,('ctrl_mean', 'exp_mean')],
                          SKNSH.loc[:,('ctrl_mean', 'exp_mean')]], axis=1)

to_attach_df.columns = ['ID',
                        'K562_ctrl_mean', 'K562_exp_mean',
                        'HepG2_ctrl_mean', 'HepG2_exp_mean', 
                        'SKNSH_ctrl_mean', 'SKNSH_exp_mean']

In [55]:
to_attach_df

,ID,K562_ctrl_mean,K562_exp_mean,HepG2_ctrl_mean,HepG2_exp_mean,SKNSH_ctrl_mean,SKNSH_exp_mean
0,1:10437778:C:T:R:wC,510.066273,3982.985875,510.066273,12014.746860,510.066273,4205.561859
1,1:10451799:C:T:R:wC,619.517011,663.645365,619.517011,648.545190,619.517011,411.699545
2,1:110198727:C:G:R:wC,38.926483,3945.704881,38.926483,1313.545467,38.926483,4110.209842
3,1:110881742:NA:NA,800.051571,23837.898516,800.051571,7711.991790,800.051571,26208.257365
4,1:111838694:C:T:R:wC,383.147871,22963.814407,383.147871,12758.700937,383.147871,40103.467033
...,...,...,...,...,...,...,...
120052,X:24893812:NA:NA,1216.543339,1320.653224,1216.543339,1207.243440,1216.543339,1555.153539
120053,X:39110157:NA:NA,718.753085,911.977644,718.753085,799.550943,718.753085,810.985256
120054,X:49154578:NA:NA,919.641399,1002.931530,919.641399,1010.359615,919.641399,1095.392076
120055,X:64815540:NA:NA,1642.172830,37378.172560,1642.172830,14984.374689,1642.172830,77452.901445


In [56]:
boda2_df = boda2_df.merge(to_attach_df, how='outer', on='ID')

In [59]:
boda2_df

,ID,sequence,K562_pred,HepG2_pred,SKNSH_pred,K562_pred_aggreg,HepG2_pred_aggreg,SKNSH_pred_aggreg,project,K562_signal,K562_lfcSE,HepG2_signal,HepG2_lfcSE,SKNSH_signal,SKNSH_lfcSE,method,target_cell,round,max_cell_pred,max_cell_signal,max_non_target_cell,max_non_target_lfc,min_non_target_cell,min_non_target_lfc,target_cell_lfc,mean_non_target_lfc,OverMean,OverMax,contrib_K562,contrib_HepG2,contrib_SKNSH,K562_ctrl_mean,K562_exp_mean,HepG2_ctrl_mean,HepG2_exp_mean,SKNSH_ctrl_mean,SKNSH_exp_mean
0,20211210_200256__612928__5::fsp_uc__hepg2__0,CAATCTGTTCCCACCATCAGGTTACTGGTTAATCATTAACCAGGGC...,-0.055265,14.231705,0.080212,0.140002,14.336414,0.227027,BODA:hepg2,2.520816,0.316205,5.287727,0.309186,0.605723,0.208455,fsp_uc,hepg2,0,hepg2,hepg2,k562,2.520816,sknsh,0.605723,5.287727,1.563270,3.724457,2.766911,[ 2.21043210e-02 4.64022532e-03 1.00994157e-...,[ 1.1422859e-01 2.5873125e-01 2.2765625e-01 ...,[ 0.004331928 0.07783466 0.06386908 ...,450.265971,2584.983789,450.265971,17588.102886,450.265971,684.911429
1,20211210_200256__612928__12::fsp_uc__hepg2__0,GGAGCTCTTTCAATGCCAGGCAGGCCCGTGGTGCAGTAAAGGCCTG...,-0.019169,19.914394,0.777519,-0.011601,18.561650,0.767500,BODA:hepg2,0.609272,0.285060,6.402153,0.301327,0.271218,0.340279,fsp_uc,hepg2,0,hepg2,hepg2,k562,0.609272,sknsh,0.271218,6.402153,0.440245,5.961907,5.792880,[-0.00856874 -0.012687626 0.03335264 ...,[-2.79185679e-02 2.33078538e-03 2.57074833e-...,[-0.017693618 -0.015454747 0.01019545 ...,186.544915,285.193149,186.544915,15770.229152,186.544915,224.886662
2,20211210_200256__612928__47::fsp_uc__hepg2__0,GGTTCAAAGTCCAGTCACAGTCCAAAGTCCACTCTTTGTCCAAGAG...,-0.146010,12.227689,-0.400996,-0.034775,10.825762,-0.337030,BODA:hepg2,1.486130,0.271874,4.276053,0.366801,-0.578350,0.241865,fsp_uc,hepg2,0,hepg2,hepg2,k562,1.486130,sknsh,-0.578350,4.276053,0.453890,3.822163,2.789924,[-5.54974526e-02 -2.83153541e-02 -1.36791170e-...,[ 0.041962117 0.2561933 0.058943637 0....,[-4.9518015e-02 -2.5722878e-02 -4.6560772e-02 ...,914.631030,2563.392133,914.631030,17720.211245,914.631030,612.217031
3,20211210_200256__612928__69::fsp_uc__hepg2__0,TAATAGTATGTCATCGTAATATGCGGACTTTGAACCGTGTACCAGG...,1.216181,18.812680,1.589409,1.416824,17.606884,1.713512,BODA:hepg2,2.440630,0.219559,5.615395,0.313984,1.063669,0.230603,fsp_uc,hepg2,0,hepg2,hepg2,k562,2.440630,sknsh,1.063669,5.615395,1.752150,3.863246,3.174765,[ 0.16365409 0.0731409 0.114835456 ...,[ 1.40735984e-01 1.30295590e-01 1.24457464e-...,[ 1.11630686e-01 9.51312631e-02 1.00722231e-...,141.922642,771.612103,141.922642,6958.102060,141.922642,296.614753
4,20211210_200256__612928__93::fsp_uc__hepg2__0,AACACGGGCACGGGGGGGCGCTGTTTGAGTTTAAAAACGATCTAGG...,1.832291,18.612158,1.411221,2.409507,17.280722,2.059474,BODA:hepg2,3.765216,0.570148,4.683140,0.612207,0.171043,2.165178,fsp_uc,hepg2,0,hepg2,hepg2,k562,3.765216,sknsh,0.171043,4.683140,1.968129,2.715011,0.917925,[ 0.017253371 0.03226251 -0.008712402 ...,[ 0.04122336 0.040660232 -0.0046606357 ...,[ 0.05285579 0.042130962 -0.035606854 ...,6.626839,90.495400,6.626839,169.731161,6.626839,7.443428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120052,ORF54316:2538:1300-1500,CTCGGAGTTCAAGAAAATCATCTCACACGGCCGTCTCAGACCCTTC...,-0.119510,1.166959,-0.200713,-0.158865,1.282273,-0.275202,MPRA:ORF_negCtrl,-0.315267,0.384919,0.591628,0.363142,0.068697,0.275408,NaN,NaN,0,hepg2,hepg2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,240.760540,195.296914,240.760540,362.833793,240.760540,252.652472
120053,ORF53130:1455:400-600,GCAATCCTACCCAGACAGATGCAGACAGCGACCTGGTGGGGGATGT...,-0.301822,-0.577524,-0.384271,-0.247987,-0.644892,-0.367910,MPRA:ORF_negCtrl,-0.134821,0.178379,-0.359782,0.261176,-0.022281,0.200300,NaN,NaN,0,k562,sknsh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,870.259457,793.895718,870.259457,678.145587,870.259457,856.918431
120054,ORF55877:2391:1900-2100,CCTATGCTGCCTCCTATGCCCTTGTCATCATTGCCATTTTTGAGCT...,1.219841,1.099753,0

In [62]:
new_column_order = ['ID', 'sequence',  'project', 'method', 'round',
                    'K562_signal', 'HepG2_signal', 'SKNSH_signal',
                    'K562_lfcSE', 'HepG2_lfcSE', 'SKNSH_lfcSE',
                    'K562_ctrl_mean', 'HepG2_ctrl_mean', 'SKNSH_ctrl_mean',
                    'K562_exp_mean', 'HepG2_exp_mean', 'SKNSH_exp_mean',             
                    'K562_pred', 'HepG2_pred', 'SKNSH_pred',
                    'K562_pred_aggreg', 'HepG2_pred_aggreg', 'SKNSH_pred_aggreg',
                    'max_cell_pred', 'max_cell_signal',
                    'target_cell', 'target_cell_lfc',
                    'max_non_target_cell', 'max_non_target_lfc',
                    'min_non_target_cell', 'min_non_target_lfc',
                    'mean_non_target_lfc',
                    'OverMean', 'OverMax',
                    'contrib_K562', 'contrib_HepG2', 'contrib_SKNSH']

In [64]:
boda2_df = boda2_df[new_column_order]

In [66]:
boda2_df = boda2_df.rename(columns={'max_cell_pred':'pred_max_cell', 'max_cell_signal':'signal_max_cell'})

In [67]:
boda2_df

,ID,sequence,project,method,round,K562_signal,HepG2_signal,SKNSH_signal,K562_lfcSE,HepG2_lfcSE,SKNSH_lfcSE,K562_ctrl_mean,HepG2_ctrl_mean,SKNSH_ctrl_mean,K562_exp_mean,HepG2_exp_mean,SKNSH_exp_mean,K562_pred,HepG2_pred,SKNSH_pred,K562_pred_aggreg,HepG2_pred_aggreg,SKNSH_pred_aggreg,pred_max_cell,signal_max_cell,target_cell,target_cell_lfc,max_non_target_cell,max_non_target_lfc,min_non_target_cell,min_non_target_lfc,mean_non_target_lfc,OverMean,OverMax,contrib_K562,contrib_HepG2,contrib_SKNSH
0,20211210_200256__612928__5::fsp_uc__hepg2__0,CAATCTGTTCCCACCATCAGGTTACTGGTTAATCATTAACCAGGGC...,BODA:hepg2,fsp_uc,0,2.520816,5.287727,0.605723,0.316205,0.309186,0.208455,450.265971,450.265971,450.265971,2584.983789,17588.102886,684.911429,-0.055265,14.231705,0.080212,0.140002,14.336414,0.227027,hepg2,hepg2,hepg2,5.287727,k562,2.520816,sknsh,0.605723,1.563270,3.724457,2.766911,[ 2.21043210e-02 4.64022532e-03 1.00994157e-...,[ 1.1422859e-01 2.5873125e-01 2.2765625e-01 ...,[ 0.004331928 0.07783466 0.06386908 ...
1,20211210_200256__612928__12::fsp_uc__hepg2__0,GGAGCTCTTTCAATGCCAGGCAGGCCCGTGGTGCAGTAAAGGCCTG...,BODA:hepg2,fsp_uc,0,0.609272,6.402153,0.271218,0.285060,0.301327,0.340279,186.544915,186.544915,186.544915,285.193149,15770.229152,224.886662,-0.019169,19.914394,0.777519,-0.011601,18.561650,0.767500,hepg2,hepg2,hepg2,6.402153,k562,0.609272,sknsh,0.271218,0.440245,5.961907,5.792880,[-0.00856874 -0.012687626 0.03335264 ...,[-2.79185679e-02 2.33078538e-03 2.57074833e-...,[-0.017693618 -0.015454747 0.01019545 ...
2,20211210_200256__612928__47::fsp_uc__hepg2__0,GGTTCAAAGTCCAGTCACAGTCCAAAGTCCACTCTTTGTCCAAGAG...,BODA:hepg2,fsp_uc,0,1.486130,4.276053,-0.578350,0.271874,0.366801,0.241865,914.631030,914.631030,914.631030,2563.392133,17720.211245,612.217031,-0.146010,12.227689,-0.400996,-0.034775,10.825762,-0.337030,hepg2,hepg2,hepg2,4.276053,k562,1.486130,sknsh,-0.578350,0.453890,3.822163,2.789924,[-5.54974526e-02 -2.83153541e-02 -1.36791170e-...,[ 0.041962117 0.2561933 0.058943637 0....,[-4.9518015e-02 -2.5722878e-02 -4.6560772e-02 ...
3,20211210_200256__612928__69::fsp_uc__hepg2__0,TAATAGTATGTCATCGTAATATGCGGACTTTGAACCGTGTACCAGG...,BODA:hepg2,fsp_uc,0,2.440630,5.615395,1.063669,0.219559,0.313984,0.230603,141.922642,141.922642,141.922642,771.612103,6958.102060,296.614753,1.216181,18.812680,1.589409,1.416824,17.606884,1.713512,hepg2,hepg2,hepg2,5.615395,k562,2.440630,sknsh,1.063669,1.752150,3.863246,3.174765,[ 0.16365409 0.0731409 0.114835456 ...,[ 1.40735984e-01 1.30295590e-01 1.24457464e-...,[ 1.11630686e-01 9.51312631e-02 1.00722231e-...
4,20211210_200256__612928__93::fsp_uc__hepg2__0,AACACGGGCACGGGGGGGCGCTGTTTGAGTTTAAAAACGATCTAGG...,BODA:hepg2,fsp_uc,0,3.765216,4.683140,0.171043,0.570148,0.612207,2.165178,6.626839,6.626839,6.626839,90.495400,169.731161,7.443428,1.832291,18.612158,1.411221,2.409507,17.280722,2.059474,hepg2,hepg2,hepg2,4.683140,k562,3.765216,sknsh,0.171043,1.968129,2.715011,0.917925,[ 0.017253371 0.03226251 -0.008712402 ...,[ 0.04122336 0.040660232 -0.0046606357 ...,[ 0.05285579 0.042130962 -0.035606854 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120052,ORF54316:2538:1300-1500,CTCGGAGTTCAAGAAAATCATCTCACACGGCCGTCTCAGACCCTTC...,MPRA:ORF_negCtrl,NaN,0,-0.315267,0.591628,0.068697,0.384919,0.363142,0.275408,240.760540,240.760540,240.760540,195.296914,362.833793,252.652472,-0.119510,1.166959,-0.200713,-0.158865,1.282273,-0.275202,hepg2,hepg2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120053,ORF53130:1455:400-600,GCAATCCTACCCAGACAGATGCAGACAGCGACCTGGTGGGGGATGT...,MPRA:ORF_negCtrl,NaN,0,-0.134821,-0.359782,-0.022281,0.178379,0.261176,0.200300,870.259457,870.259457,870.259457,793.895718,678.145587,856.918431,-0.301822,-0.577524,-0.384271,-0.247987,-0.644892,-0.367910,k562,sknsh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120054,ORF55877:2391:1900-2100,CCTATGCTGCCTCCTATGCCCTTGTCATCATTGCCATTTTTGAGCT...,MPRA:ORF_negCtrl,Na

In [71]:
row_filter = (boda2_df['method'] == 'genome_scan')
boda2_df.loc[row_filter, 'method'] = 'gs'

In [72]:
boda2_df.method.unique()

array(['fsp_uc', 'hmc', 'sa', 'sa_uc', 'sa_rep', 'al', 'al_uc', 'fsp',
       'DHS', 'gs', nan], dtype=object)

In [73]:
#boda2_df.to_csv('/Users/castrr/Dropbox (JAX)/Datasets/BODA2/BODA2_MPRA_results_pred_contributions_v1.txt', index=None, sep=' ')

In [ ]:
# out_df = boda2_df[['ID', 'sequence', 'contrib_K562', 'contrib_HepG2', 'contrib_SKNSH']].copy().reset_index(drop=True)
# out_df.to_csv('/Users/castrr/Dropbox (JAX)/Datasets/BODA2/BODA2_MPRA_contributions.txt', index=None, sep=' ')